In [1]:
import json

# Load configuration from JSON
config_file = "pipeline_config.json"

# Read the N_Trials value
try:
    with open(config_file, "r") as f:
        config = json.load(f)
        N_TRIALS = config.get("N_Trials", 1)  # Default to 1 if not found
except FileNotFoundError:
    print(f"Config file {config_file} not found. Using default values.")
    N_TRIALS = 1
except json.JSONDecodeError:
    print(f"Error parsing {config_file}. Using default values.")
    N_TRIALS = 1


In [2]:
!pip install shap
!pip install xgboost

In [3]:
import pandas as pd
import numpy as np

import boto3
from pyathena import connect
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup, FeatureDefinition, FeatureTypeEnum
from sagemaker.session import Session
from sagemaker import get_execution_role

from pyathena import connect

import time
import shap
import json
import joblib
import io

import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, precision_score, recall_score

from itertools import combinations
import pickle
import warnings

import model_methods


# S3 and Athena details
bucket_name = "group3-project-bucket"
database_name = "group_project_db"
table_name = "hospital_readmissions"
s3_output = f"s3://{bucket_name}/athena-results/"
region = "us-east-1"
s3_client = boto3.client("s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml


sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Find the latest feature store to generate a new model!

In [4]:
glue_client = boto3.client("glue")

# List databases in AWS Glue
response = glue_client.get_databases()
print("\nAvailable Databases in Glue:")
for db in response["DatabaseList"]:
    print(f"- {db['Name']}")

# List tables in the `sagemaker_featurestore` database (if it exists)
database_name = "sagemaker_featurestore"

try:
    response = glue_client.get_tables(DatabaseName=database_name)
    print(f"\nTables in `{database_name}` database:")
    for table in response["TableList"]:
        print(f"- {table['Name']}")
except glue_client.exceptions.EntityNotFoundException:
    print(f"\nDatabase `{database_name}` not found in Glue.")


Available Databases in Glue:
- default
- group_project_db
- sagemaker_featurestore

Tables in `sagemaker_featurestore` database:
- hospital_readmissions_features_1740354579
- hospital_readmissions_features_1740867183_1740867183
- hospital_readmissions_features_1740867247_1740867247
- hospital_readmissions_features_1740867455_1740867455
- hospital_readmissions_features_1740867741_1740867741
- hospital_readmissions_features_1740868107_1740868107
- hospital_readmissions_features_1740868421_1740868421
- hospital_readmissions_features_1740868692_1740868692
- hospital_readmissions_features_1740869180_1740869180
- hospital_readmissions_features_1740869682_1740869682
- hospital_readmissions_features_1740871225_1740871225
- hospital_readmissions_features_1740871778_1740871779
- hospital_readmissions_features_1740872136_1740872136
- hospital_readmissions_features_1740950588_1740950588
- hospital_readmissions_features_1740951018_1740951018


Query Athenta Tables for Data Splitting

In [5]:
# Query the feature store in Athena

latest_table = response["TableList"][-1]
query = f"""
SELECT * 
FROM "sagemaker_featurestore"."{latest_table["Name"]}"
"""

# Connect to Athena
connection = connect(
    s3_staging_dir=f"s3://{bucket_name}/athena-results/",
    region_name="us-east-1"
)

# Retrieve all feature data
df = pd.read_sql(query, connection)

/tmp/ipykernel_837/530253379.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [6]:
df = df.drop(columns=["event_time", "write_time", "api_invocation_time", "is_deleted"])
display(df.head())


print("Base Features: ", len(df.columns))
print("Data Samples: ", len(df))

categorical_columns, num_cats = model_methods.get_categorical_columns_from_s3(bucket_name, s3_client)
print("Categorical columns: ", categorical_columns)
print("Num classes per category: ", num_cats)

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,a1ctest,change,diabetes_med,readmitted
0,1,5,45,0,18,0,0,0,4,0,0,0,1,1,1,1,1
1,3,2,73,0,14,0,0,0,0,7,0,1,1,1,0,1,0
2,4,5,60,0,16,0,2,0,3,6,3,6,1,1,1,1,0
3,2,1,46,0,8,0,0,0,1,7,0,6,1,1,0,1,0
4,2,1,42,0,7,0,0,0,3,6,0,7,1,1,0,1,0


Base Features:  17
Data Samples:  16459
Identified categorical columns: ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Category counts per categorical column: {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}
Categorical columns:  ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Num classes per category:  {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}


Train an XGB model raw, use SHAP to visual feature importance.

In [7]:
# Split data into train (50%), test (10%), production (40%)
train_df, temp_df = train_test_split(df, test_size=0.5, random_state=42)
test_df, prod_df = train_test_split(temp_df, test_size=0.8, random_state=42)

# Separate features and target variable
target_column = "readmitted"
X_train = train_df.drop(columns=[target_column])
y_train = train_df[target_column]
X_test = test_df.drop(columns=[target_column])
y_test = test_df[target_column]

# Train XGBoost model on production dataset
dmatrix_train = xgb.DMatrix(X_train, label=y_train)
params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "seed": 42
}
model = xgb.train(params, dmatrix_train, num_boost_round=100)

def eval_model(model, X_test, y_test):
    """
    Evaluates the XGBoost model and returns evaluation metrics.
    Ensures all relevant metrics are logged for the model card.
    """
    # Convert test data into DMatrix
    dmatrix_test = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
    
    # Make predictions
    y_pred_proba = model.predict(dmatrix_test)
    y_pred = (y_pred_proba >= 0.5).astype(int)  # Convert probabilities to binary predictions
    
    # Compute evaluation metrics
    test_log_loss = log_loss(y_test, y_pred_proba)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_auc = roc_auc_score(y_test, y_pred_proba)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)

    # Print evaluation metrics
    print(f"✅ Test Log Loss: {test_log_loss:.4f}")
    print(f"✅ Test Accuracy: {test_accuracy:.4f}")
    print(f"✅ Test AUC: {test_auc:.4f}")
    print(f"✅ Test Precision: {test_precision:.4f}")
    print(f"✅ Test Recall: {test_recall:.4f}")

    return test_accuracy, test_auc, test_precision, test_recall


eval_model(model, X_test, y_test)

✅ Test Log Loss: 0.7097
✅ Test Accuracy: 0.5923
✅ Test AUC: 0.6200
✅ Test Precision: 0.5795
✅ Test Recall: 0.5331


(0.5923450789793439,
 0.6200100597668501,
 0.5795297372060858,
 0.5330788804071247)

Extract Important Features from XGBoost Model.. create interacting features from statistitically most important features.

In [8]:
X_train_final, X_test_final = model_methods.shap_feature_engineering(model, X_train, X_test, bucket_name, s3_client)
display(X_train_final.head())
display(X_test_final.head())

print(X_train_final.dtypes)

Identified categorical columns: ['glucose_test', 'A1Ctest', 'age', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'change', 'diabetes_med', 'readmitted']
Category counts per categorical column: {'glucose_test': 3, 'A1Ctest': 3, 'age': 6, 'medical_specialty': 7, 'diag_1': 8, 'diag_2': 8, 'diag_3': 8, 'change': 2, 'diabetes_med': 2, 'readmitted': 2}


Top Features: ['n_inpatient', 'n_lab_procedures', 'n_medications', 'medical_specialty', 'n_outpatient', 'time_in_hospital', 'n_procedures', 'diag_3', 'diag_2', 'age', 'diag_1', 'n_emergency', 'a1ctest', 'change', 'diabetes_med', 'glucose_test']
Skipping interaction: n_inpatient × medical_specialty (Categorical × Numeric)
Skipping interaction: n_lab_procedures × medical_specialty (Categorical × Numeric)
Skipping interaction: n_medications × medical_specialty (Categorical × Numeric)
Skipping interaction: medical_specialty × n_outpatient (Categorical × Numeric)
Interaction features saved to s3://group3-project-bucket/config/interaction_features.json


,n_inpatient,n_lab_procedures,n_medications,medical_specialty,n_outpatient,time_in_hospital,n_procedures,diag_3,diag_2,age,...,a1ctest,change,diabetes_med,glucose_test,n_inpatient_x_n_lab_procedures,n_inpatient_x_n_medications,n_inpatient_x_n_outpatient,n_lab_procedures_x_n_medications,n_lab_procedures_x_n_outpatient,n_medications_x_n_outpatient
1603,0,57,12,3,0,1,0,6,1,3,...,1,0,0,1,0,0,0,684,0,0
8014,1,53,21,3,0,6,0,1,6,3,...,1,1,1,1,53,21,0,1113,0,0
4065,1,73,20,1,0,10,3,6,0,4,...,1,1,1,1,73,20,0,1460,0,0
1294,0,51,18,5,0,6,2,0,7,4,...,1,1,1,1,0,0,0,918,0,0
15494,0,44,11,3,0,6,0,1,0,3,...,2,1,1,1,0,0,0,484,0,0


,n_inpatient,n_lab_procedures,n_medications,medical_specialty,n_outpatient,time_in_hospital,n_procedures,diag_3,diag_2,age,...,a1ctest,change,diabetes_med,glucose_test,n_inpatient_x_n_lab_procedures,n_inpatient_x_n_medications,n_inpatient_x_n_outpatient,n_lab_procedures_x_n_medications,n_lab_procedures_x_n_outpatient,n_medications_x_n_outpatient
5064,1,34,5,1,0,1,0,6,1,1,...,1,0,0,1,34,5,0,170,0,0
6561,4,54,12,4,5,3,0,6,6,4,...,1,0,0,1,216,48,20,648,270,60
14458,3,32,21,5,0,4,2,0,0,3,...,1,0,0,1,96,63,0,672,0,0
13293,0,39,19,4,0,4,0,7,0,3,...,1,0,1,1,0,0,0,741,0,0
10114,0,75,31,1,0,9,1,6,0,2,...,0,1,1,1,0,0,0,2325,0,0


n_inpatient                         int64
n_lab_procedures                    int64
n_medications                       int64
medical_specialty                   int64
n_outpatient                        int64
time_in_hospital                    int64
n_procedures                        int64
diag_3                              int64
diag_2                              int64
age                                 int64
diag_1                              int64
n_emergency                         int64
a1ctest                             int64
change                              int64
diabetes_med                        int64
glucose_test                        int64
n_inpatient_x_n_lab_procedures      int64
n_inpatient_x_n_medications         int64
n_inpatient_x_n_outpatient          int64
n_lab_procedures_x_n_medications    int64
n_lab_procedures_x_n_outpatient     int64
n_medications_x_n_outpatient        int64
dtype: object


## Save Production Data to an s3 bucket for Later

In [9]:
X_prod = prod_df.drop(columns=[target_column])
y_prod = prod_df[target_column]

X_prod_final = model_methods.apply_interaction_features(X_prod, bucket_name, s3_client)
print(X_prod_final.columns)

Index(['age', 'time_in_hospital', 'n_lab_procedures', 'n_procedures',
       'n_medications', 'n_outpatient', 'n_inpatient', 'n_emergency',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'glucose_test',
       'a1ctest', 'change', 'diabetes_med', 'n_inpatient_x_n_lab_procedures',
       'n_inpatient_x_n_medications', 'n_inpatient_x_n_outpatient',
       'n_lab_procedures_x_n_medications', 'n_lab_procedures_x_n_outpatient',
       'n_medications_x_n_outpatient'],
      dtype='object')


In [10]:
# Define S3 bucket and file path
def save_to_prod(df, name):
    s3_filename = f"production_data/{name}.csv"
    s3_uri = f"s3://{bucket_name}/{s3_filename}"
    
    # Convert DataFrame to CSV in memory
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    
    # Upload CSV file to S3
    s3_client.put_object(
        Bucket=bucket_name,
        Key=s3_filename,
        Body=csv_buffer.getvalue()
    )
    
    print(f"Data saved to S3: {s3_uri}")

save_to_prod(X_prod_final, "X_prod_final")
save_to_prod(y_prod, "y_prod")
save_to_prod(X_train_final, "X_train_final")
save_to_prod(y_train, "y_train")

Data saved to S3: s3://group3-project-bucket/production_data/X_prod_final.csv
Data saved to S3: s3://group3-project-bucket/production_data/y_prod.csv
Data saved to S3: s3://group3-project-bucket/production_data/X_train_final.csv
Data saved to S3: s3://group3-project-bucket/production_data/y_train.csv


Bayesian Optimization Procedure to find best XGB model

In [11]:
!pip install optuna

In [12]:
import optuna
from sklearn.metrics import roc_auc_score

def xgb_objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'seed': 42
    }
    
    dtrain = xgb.DMatrix(X_train_final, label=y_train, enable_categorical=True)
    dtest = xgb.DMatrix(X_test_final, label=y_test, enable_categorical=True)
    model = xgb.train(params, dtrain, num_boost_round=100)
    preds = model.predict(dtest)
    
    return roc_auc_score(y_test, preds)

study = optuna.create_study(direction="maximize")
study.optimize(xgb_objective, n_trials=N_TRIALS)

best_params = study.best_params
best_params['max_depth'] = int(best_params['max_depth'])

[I 2025-03-02 21:32:52,163] A new study created in memory with name: no-name-5a4e90e0-aa3a-4f4a-bd64-445cbb1b2e57


[I 2025-03-02 21:32:52,393] Trial 0 finished with value: 0.6370214213858809 and parameters: {'learning_rate': 0.18960849349554365, 'max_depth': 5, 'min_child_weight': 6.34158821166558, 'colsample_bytree': 0.35158547345322544, 'subsample': 0.8292140641566549}. Best is trial 0 with value: 0.6370214213858809.


[I 2025-03-02 21:32:52,596] Trial 1 finished with value: 0.640515710988816 and parameters: {'learning_rate': 0.1359193727263146, 'max_depth': 6, 'min_child_weight': 2.6450293002099037, 'colsample_bytree': 0.36802521618836925, 'subsample': 0.8407364175263596}. Best is trial 1 with value: 0.640515710988816.


[I 2025-03-02 21:32:52,818] Trial 2 finished with value: 0.6602861116042369 and parameters: {'learning_rate': 0.050869536160763756, 'max_depth': 6, 'min_child_weight': 9.295225637797456, 'colsample_bytree': 0.30929093568101634, 'subsample': 0.7039251258132115}. Best is trial 2 with value: 0.6602861116042369.


[I 2025-03-02 21:32:52,977] Trial 3 finished with value: 0.6382759334871886 and parameters: {'learning_rate': 0.2705152074840462, 'max_depth': 3, 'min_child_weight': 7.536937318052674, 'colsample_bytree': 0.8596631363634335, 'subsample': 0.8017182510802531}. Best is trial 2 with value: 0.6602861116042369.


...

[I 2025-03-02 21:36:29,215] Trial 995 finished with value: 0.6639091070477544 and parameters: {'learning_rate': 0.05535177388228427, 'max_depth': 4, 'min_child_weight': 9.221608999877374, 'colsample_bytree': 0.38203991776272933, 'subsample': 0.5171802074561753}. Best is trial 959 with value: 0.6729821291200662.


[I 2025-03-02 21:36:29,532] Trial 996 finished with value: 0.6579087519971596 and parameters: {'learning_rate': 0.057980017920436826, 'max_depth': 9, 'min_child_weight': 9.070169657888988, 'colsample_bytree': 0.35493983015249664, 'subsample': 0.5413750027147914}. Best is trial 959 with value: 0.6729821291200662.


[I 2025-03-02 21:36:29,835] Trial 997 finished with value: 0.6507337712290668 and parameters: {'learning_rate': 0.06614570258912121, 'max_depth': 7, 'min_child_weight': 9.502593160788154, 'colsample_bytree': 0.8140815364682404, 'subsample': 0.5252256341344215}. Best is trial 959 with value: 0.6729821291200662.


[I 2025-03-02 21:36:30,064] Trial 998 finished with value: 0.6651887685661872 and parameters: {'learning_rate': 0.05442998215722557, 'max_depth': 4, 'min_child_weight': 9.547445084695138, 'colsample_bytree': 0.36959271885646877, 'subsample': 0.5005125832413029}. Best is trial 959 with value: 0.6729821291200662.


[I 2025-03-02 21:36:30,296] Trial 999 finished with value: 0.6701476418723001 and parameters: {'learning_rate': 0.065631885509412, 'max_depth': 4, 'min_child_weight': 9.99583599372239, 'colsample_bytree': 0.34714424814586137, 'subsample': 0.5164166328434655}. Best is trial 959 with value: 0.6729821291200662.


In [13]:
# Train the best model
dmatrix_train = xgb.DMatrix(X_train_final, label=y_train, enable_categorical=True)
model = xgb.train(best_params, dmatrix_train, num_boost_round=500)

In [14]:
# Visualize Optuna Trials
fig = optuna.visualization.matplotlib.plot_optimization_history(study)
plt.savefig("figures/optuna_optimization_history.png")
plt.close()

fig = optuna.visualization.matplotlib.plot_param_importances(study)
plt.savefig("figures/optuna_param_importance.png")
plt.close()

/tmp/ipykernel_837/737199464.py:2: ExperimentalWarning: plot_optimization_history is experimental (supported from v2.2.0). The interface can change in the future.
  fig = optuna.visualization.matplotlib.plot_optimization_history(study)


/tmp/ipykernel_837/737199464.py:6: ExperimentalWarning: plot_param_importances is experimental (supported from v2.2.0). The interface can change in the future.
  fig = optuna.visualization.matplotlib.plot_param_importances(study)


In [15]:
explainer = shap.Explainer(model, X_test_final)
shap_values = explainer(X_test_final)

# Save SHAP summary plot
plt.figure()
shap.summary_plot(shap_values, X_test_final, show=False)
plt.savefig("figures/final_shap_summary.png")
plt.close()

# Save SHAP dependence plot for the first feature
plt.figure()
shap.dependence_plot(0, shap_values.values, X_test_final, show=False)
plt.savefig("figures/final_shap_dependence_0.png")
plt.close()

 53%|===========         | 866/1646 [00:11<00:09]       

 57%|===========         | 932/1646 [00:12<00:09]       

 61%|============        | 1005/1646 [00:13<00:08]       

 66%|=============       | 1094/1646 [00:14<00:07]       

 72%|==============      | 1187/1646 [00:15<00:05]       

 78%|================    | 1285/1646 [00:16<00:04]       

 84%|=================   | 1379/1646 [00:17<00:03]       

 90%|==================  | 1477/1646 [00:18<00:02]       

 95%|=================== | 1557/1646 [00:19<00:01]       

<Figure size 640x480 with 0 Axes>

In [16]:
print("TEST")
acc, roc, precision, recall = eval_model(model, X_test_final, y_test)

print("TRAIN")
acc_train, roc_train, precision_train, recall_train = eval_model(model, X_train_final, y_train)

TEST
✅ Test Log Loss: 0.6669
✅ Test Accuracy: 0.6063
✅ Test AUC: 0.6430
✅ Test Precision: 0.6015
✅ Test Recall: 0.5204
TRAIN
✅ Test Log Loss: 0.5739
✅ Test Accuracy: 0.7119
✅ Test AUC: 0.7875
✅ Test Precision: 0.7464
✅ Test Recall: 0.5975


In [17]:
# Save the model
model.save_model("models/tuned_xgboost_model.model")
print("Model saved to models/tuned_xgboost_model.model")

Model saved to models/tuned_xgboost_model.model


/opt/conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:37:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880431132/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


## PUT XGBOOST MODEL IN MODEL STORE

In [18]:
import tarfile
import json

region = boto3.Session().region_name
role = get_execution_role()
sagemaker_session = sagemaker.Session()

best_params = study.best_params

# ✅ SageMaker client
sm_client = boto3.client("sagemaker", region_name=region)
s3_client = boto3.client("s3", region_name=region)

# ✅ **Model Package Group Name (Unique Identifier)**
model_package_group_name = "xgboost-hospital-readmissions-" + str(int(time.time()))
with open("pipeline_model_package.json", "w") as json_file:
        json.dump({
            "model_package": model_package_group_name
        },json_file, indent=4)

model_package_group_description = "XGBoost model for predicting hospital readmissions."


# ✅ **Create Model Package Group**
create_model_package_group_response = sm_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription=model_package_group_description,
)

print("✅ ModelPackageGroup Arn:", create_model_package_group_response["ModelPackageGroupArn"])

# ✅ **Save the trained XGBoost model**
joblib.dump(model, "model.joblib")

# ✅ **Save the model as a compressed `.tar.gz` file**
tar_filename = "model.tar.gz"
with tarfile.open(tar_filename, "w:gz") as tar:
    tar.add("model.joblib")

# ✅ **Upload model to S3**
timestamp = int(time.time())
prefix = "hospital-readmissions-xgboost"
s3_key = f"{prefix}/model-{timestamp}.tar.gz"
model_s3_uri = f"s3://{bucket_name}/{s3_key}"

s3_client.upload_file(tar_filename, bucket_name, s3_key)
print(f"✅ Model artifact uploaded to: {model_s3_uri}")

# ✅ **Define SageMaker Model Package**
model_package_name = "xgboost-hospital-readmissions-package-" + str(int(time.time()))
xgboost_image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1"

inference_spec = {
    "Containers": [
        {
            "Image": xgboost_image_uri,
            "ModelDataUrl": model_s3_uri,
        }
    ],
    "SupportedTransformInstanceTypes": ["ml.m5.large", "ml.m5.xlarge"],
    "SupportedRealtimeInferenceInstanceTypes": ["ml.m5.large", "ml.m5.xlarge"],
    "SupportedContentTypes": ["text/csv"],
    "SupportedResponseMIMETypes": ["text/csv", "application/json"],
}

create_model_package_response = sm_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="XGBoost model for predicting hospital readmissions.",
    InferenceSpecification=inference_spec,
    ModelApprovalStatus="PendingManualApproval",
)

print("✅ Model Package ARN:", create_model_package_response["ModelPackageArn"])

# ✅ **Describe the Model Package**
response = sm_client.describe_model_package(
    ModelPackageName=create_model_package_response["ModelPackageArn"]
)

print("Model Package Details:", response)

# ✅ **Create a Model Card**
model_card_name = f"xgboost-hospital-readmissions-card-{int(time.time())}"

# ✅ Update Model Card with **full evaluation details**
model_card_content = {
    "model_overview": {
        "model_description": "XGBoost model predicting hospital readmissions.",
        "model_creator": "Your Team",
        "model_artifact": [model_s3_uri],
        "algorithm_type": "XGBoost",
        "problem_type": "Binary Classification",
        "model_owner": "Group 3",
    },
    "intended_uses": {
        "purpose_of_model": "Predict likelihood of hospital readmission within 30 days.",
        "intended_uses": "Hospital decision support, optimizing patient care.",
        "risk_rating": "Medium",
        "explanations_for_risk_rating": "Incorrect predictions may lead to unnecessary interventions or missed readmissions.",
    },
    "business_details": {
        "business_problem": "Reducing hospital readmission rates.",
        "business_stakeholders": "Healthcare providers, insurers, hospital administrators.",
    },
    "training_details": {
        "training_observations": "Model trained on historical patient admission data.",
        "training_job_details": {
            "training_datasets": [f"sagemaker_featurestore.{latest_table['Name']}"],
            "training_environment": {
                "container_image": [xgboost_image_uri],
            },
            "training_metrics": [
                {"name": "accuracy", "value": acc_train},
                {"name": "roc_auc", "value": roc_train},
                {"name": "precision", "value": precision_train},
                {"name": "recall", "value": recall_train},
            ],
            "hyper_parameters": [
                {"name": k, "value": str(v)} for k, v in best_params.items()
            ],
        },
    },
    "evaluation_details": [
        {
            "name": "evaluation-1",
            "datasets": [f"sagemaker_featurestore.{latest_table['Name']}"],
            "metric_groups": [
                {
                    "name": "readmission-metrics",
                    "metric_data": [
                        {"name": "accuracy", "type": "number", "value": acc},
                        {"name": "roc_auc", "type": "number", "value": roc},
                        {"name": "precision", "type": "number", "value": precision},
                        {"name": "recall", "type": "number", "value": recall},
                    ],
                }
            ],
            "evaluation_observation": "Evaluated on a holdout test dataset.",
        }
    ],
    "additional_information": {
        "ethical_considerations": "Model should not be used to make final medical decisions.",
        "caveats_and_recommendations": "Should be used alongside physician assessment.",
    },
}


# ✅ **Create the Model Card**
response = sm_client.create_model_card(
    ModelCardName=model_card_name,
    Content=json.dumps(model_card_content),
    ModelCardStatus="Draft",
)

print("✅ Model Card ARN:", response["ModelCardArn"])
print("✅ Model Card Name:", model_card_name)

# ✅ **Describe the Model Card**
describe_response = sm_client.describe_model_card(ModelCardName=model_card_name)
print(describe_response)

✅ ModelPackageGroup Arn: arn:aws:sagemaker:us-east-1:321261761338:model-package-group/xgboost-hospital-readmissions-1740951464


✅ Model artifact uploaded to: s3://group3-project-bucket/hospital-readmissions-xgboost/model-1740951465.tar.gz


✅ Model Package ARN: arn:aws:sagemaker:us-east-1:321261761338:model-package/xgboost-hospital-readmissions-1740951464/1
Model Package Details: {'ModelPackageGroupName': 'xgboost-hospital-readmissions-1740951464', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:321261761338:model-package/xgboost-hospital-readmissions-1740951464/1', 'ModelPackageDescription': 'XGBoost model for predicting hospital readmissions.', 'CreationTime': datetime.datetime(2025, 3, 2, 21, 37, 45, 584000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1', 'ImageDigest': 'sha256:c764382b16cd0c921f1b2e66de8684fb999ccbd0c042c95679f0b69bc9cdd12c', 'ModelDataUrl': 's3://group3-project-bucket/hospital-readmissions-xgboost/model-1740951465.tar.gz', 'ModelDataETag': '596ac0fca13760d202b13b7fa3eec80e'}], 'SupportedTransformInstanceTypes': ['ml.m5.large', 'ml.m5.xlarge'], 'SupportedRealtimeInferenceInstanceTyp

✅ Model Card ARN: arn:aws:sagemaker:us-east-1:321261761338:model-card/xgboost-hospital-readmissions-card-1740951465
✅ Model Card Name: xgboost-hospital-readmissions-card-1740951465


{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:321261761338:model-card/xgboost-hospital-readmissions-card-1740951465', 'ModelCardName': 'xgboost-hospital-readmissions-card-1740951465', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_description": "XGBoost model predicting hospital readmissions.", "model_creator": "Your Team", "model_artifact": ["s3://group3-project-bucket/hospital-readmissions-xgboost/model-1740951465.tar.gz"], "algorithm_type": "XGBoost", "problem_type": "Binary Classification", "model_owner": "Group 3"}, "intended_uses": {"purpose_of_model": "Predict likelihood of hospital readmission within 30 days.", "intended_uses": "Hospital decision support, optimizing patient care.", "risk_rating": "Medium", "explanations_for_risk_rating": "Incorrect predictions may lead to unnecessary interventions or missed readmissions."}, "business_details": {"business_problem": "Reducing hospital readmission rates.", "business_stakeholders": "Healthcare providers, insurers, ho